In [ ]:
import os
file_path = os.path.abspath("school_safety.csv")
print(file_path)

In [ ]:
#Chrome
from selenium import webdriver 
from selenium.webdriver.common.by import By
from time import sleep
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
df = pd.read_csv(r'C:\Users\ryanb\Semester 8\DS 490 Capstone\DS 490 Report Writing\school_safety.csv')

In [ ]:
df['Address'] = np.nan

In [ ]:
options = Options()
options.executable_path = "C:/Users/ryanb/OneDrive/Desktop/Semester 8/DS 490 Data Science Capstone/chromedriver.exe"
options.add_argument("--incognito")
browser = webdriver.Chrome(options=options)

In [ ]:
browser.get("https://gps-coordinates.org/")

prev_address = None
refresh_count = 0

for index, row in tqdm(df.iterrows()):
    if pd.isnull(row['Address']):
    
        lat = str(row['LAT'])
        lon = str(row['LNG'])
        # Latitude
        inputlat = browser.find_element(By.XPATH, '/html/body/table[5]/tbody/tr/td[1]/table[1]/tbody/tr[1]/td[2]/input')
        inputlat.clear()
        inputlat.send_keys(lat)
        # Longitude
        inputlon = browser.find_element(By.XPATH, '//*[@id="longitude"]')
        inputlon.clear() 
        inputlon.send_keys(lon)
        inputbut = browser.find_element(By.XPATH, '//*[@id="btnGetAddressByCoordinates"]')
        time.sleep(3)
        inputbut.click()
        time.sleep(3)
        address_field = browser.find_element(By.ID, "address")
        address = address_field.get_attribute("value")
        print(address)
        if address == prev_address:
            print("Got same address as previous: ", address)  
            raise ValueError("Duplicate address")
        time.sleep(3)
        pd.set_option('display.max_colwidth', None)
        # Update the Address column for this row
        df.loc[index, 'Address'] = address

        refresh_count += 1
        if refresh_count == 15:
            print("Refreshing browser...")
            browser.refresh()
            refresh_count = 0

In [ ]:
df.to_csv('school_safety_addresses', index=False)